In [2]:
import sqlalchemy as sq
sq.create_engine('sqlite:///RaysPitching.db')
%load_ext sql
%sql sqlite:///RaysPitching.db

## Q1. Average Piches Per at Bat Analysis

### A. Average Pitches Per At Bat (LastPitchRays)

In [5]:
%%sql
SELECT AVG(Pitch_number) as AvgNumofPitchesPerAtBat
FROM LastPitchRays

 * sqlite:///RaysPitching.db
Done.


AvgNumofPitchesPerAtBat
3.830943141555593


### B. Average Pitches Per At Bat Home vs Away (LastPitchRays)

In [9]:
%%sql
SELECT 'Home' TypeofGame, AVG(Pitch_number) as AvgNumofPitchesPerAtBat
FROM LastPitchRays 
WHERE home_team = 'TB'
UNION 
SELECT 'Away' TypeofGame, AVG(Pitch_number) as AvgNumofPitchesPerAtBat
FROM LastPitchRays 
WHERE away_team = 'TB'

 * sqlite:///RaysPitching.db
Done.


TypeofGame,AvgNumofPitchesPerAtBat
Away,3.802380952380952
Home,3.8590559089387346


### C. Average Pitches Per At Bat Lefty vs Righty

In [10]:
%%sql
SELECT AVG(Case when batter_position = 'L' Then Pitch_number end) as LeftyatBats, 
       AVG(Case when batter_position = 'R' Then Pitch_number end) as RightyatBats
FROM LastPitchRays 

 * sqlite:///RaysPitching.db
Done.


LeftyatBats,RightyatBats
3.866182572614108,3.813953488372093


### D. Average pitches Per At Bat Left vs Righty pitcher | Each Away Team -> Partition By

In [12]:
%%sql
SELECT DISTINCT
       home_team, 
       Pitcher_position,
       AVG(Pitch_number) OVER (Partition by home_team, Pitcher_position)
FROM LastPitchRays
WHERE away_team = 'TB'


 * sqlite:///RaysPitching.db
Done.


home_team,pitcher_position,"AVG(Pitch_number) OVER (Partition by home_team, Pitcher_position)"
BAL,L,3.549019607843137
BAL,R,3.7554585152838427
BOS,L,4.08235294117647
BOS,R,3.7516339869281046
CHC,L,4.0
CHC,R,4.238095238095238
CIN,L,4.357142857142857
CIN,R,4.089743589743589
CLE,L,3.9743589743589745
CLE,R,4.328947368421052


### E. Top 3 Common Pitch for at bat 1 through 10, and total amounts (LastPitchRays)

In [18]:
%%sql
with totalpitchsequence as (
	SELECT DISTINCT
		Pitch_name,
		Pitch_number,
		count(pitch_name) OVER (Partition by Pitch_name, Pitch_number) PitchFrequency
	FROM LastPitchRays
	where Pitch_number < 11
),
pitchfrequencyrankquery as (
	SELECT 
	Pitch_name,
	Pitch_number,
	PitchFrequency,
	rank() OVER (Partition by Pitch_number order by PitchFrequency desc) PitchFrequencyRanking
FROM totalpitchsequence
)
SELECT *
FROM pitchfrequencyrankquery
WHERE PitchFrequencyRanking < 4

 * sqlite:///RaysPitching.db
Done.


Pitch_name,Pitch_number,PitchFrequency,PitchFrequencyRanking
Sinker,1,152,1
Cutter,1,149,2
4-Seam Fastball,1,147,3
4-Seam Fastball,2,184,1
Sinker,2,179,2
Changeup,2,143,3
4-Seam Fastball,3,282,1
Changeup,3,221,2
Slider,3,155,3
4-Seam Fastball,4,291,1


### F. Average Pitches Per at Bat Per Pitcher with 20+ innings | Order in descending (LastPitchRays + RaysPitchingStats)

In [22]:
%%sql
SELECT 
	RPS.Name, 
	AVG(Pitch_number) AVGPitches
FROM LastPitchRays as LPR
JOIN RaysPitchingStats as RPS ON RPS.pitcher_id = LPR.pitcher
WHERE IP >= 20
group by RPS.Name
order by AVG(Pitch_number) DESC

 * sqlite:///RaysPitching.db
Done.


Name,AVGPitches
Calvin Faucher,4.287128712871287
Matt Wisler,4.2628571428571425
Pete Fairbanks,4.045977011494253
J.P. Feyereisen,4.0
Jason Adam,3.9872340425531916
Jalen Beeks,3.9800796812749004
Colin Poche,3.9673469387755103
Shawn Armstrong,3.9615384615384617
Jeffrey Springs,3.901818181818182
Luis Patiño,3.898989898989899


## Q2. Last Pitch Analysis

### A. Count of the Last Pitches Thrown in DESC order (LastPitchRays)

In [23]:
%%sql
SELECT pitch_name, count(*) timesthrown
FROM LastPitchRays
group by pitch_name
order by count(*) desc

 * sqlite:///RaysPitching.db
Done.


pitch_name,timesthrown
4-Seam Fastball,1430
Changeup,973
Sinker,889
Slider,857
Cutter,689
Curveball,615
Sweeper,402
Other,29
Eephus,17
Knuckle Curve,12


### B. Count of the different last pitches Fastball or Offspeed (LastPitchRays)

In [24]:
%%sql
SELECT
	sum(case when pitch_name in ('4-Seam Fastball', 'Cutter') then 1 else 0 end) Fastball,
	sum(case when pitch_name NOT in ('4-Seam Fastball', 'Cutter') then 1 else 0 end) Offspeed
FROM LastPitchRays

 * sqlite:///RaysPitching.db
Done.


Fastball,Offspeed
2119,3808


### C. Percentage of the different last pitches Fastball or Offspeed (LastPitchRays)

In [25]:
%%sql
SELECT
	100 * sum(case when pitch_name in ('4-Seam Fastball', 'Cutter') then 1 else 0 end) / count(*) FastballPercent,
	100 * sum(case when pitch_name NOT in ('4-Seam Fastball', 'Cutter') then 1 else 0 end) / count(*) OffspeedPercent
FROM LastPitchRays

 * sqlite:///RaysPitching.db
Done.


FastballPercent,OffspeedPercent
35,64


### D. Top 5 Most common last pitch for a Relief Pitcher vs Starting Pitcher (LastPitchRays + RaysPitchingStats)

In [26]:
%%sql
SELECT *
FROM (
	SELECT 
		a.POS, 
		a.pitch_name,
		a.timesthrown,
		RANK() OVER (Partition by a.POS Order by a.timesthrown desc) PitchRank
	FROM (
		SELECT RPS.POS, LPR.pitch_name, count(*) timesthrown
		FROM LastPitchRays as LPR
		JOIN RaysPitchingStats as RPS ON RPS.pitcher_id = LPR.pitcher
		group by RPS.POS, LPR.pitch_name
	) a
)b
WHERE b.PitchRank < 6

 * sqlite:///RaysPitching.db
Done.


POS,pitch_name,timesthrown,PitchRank
RP,4-Seam Fastball,711,1
RP,Sinker,655,2
RP,Slider,547,3
RP,Changeup,474,4
RP,Cutter,307,5
SP,4-Seam Fastball,719,1
SP,Changeup,499,2
SP,Curveball,404,3
SP,Cutter,382,4
SP,Slider,310,5


## Q3. Homerun analysis

### A. What pitches have given up the most HRs (LastPitchRays)

In [27]:
%%sql
SELECT pitch_name, count(*) HRs
FROM LastPitchRays
where events = 'home_run'
group by pitch_name
order by count(*) desc

 * sqlite:///RaysPitching.db
Done.


pitch_name,HRs
4-Seam Fastball,50
Slider,26
Sinker,22
Cutter,22
Curveball,20
Changeup,15
Sweeper,11
Other,3
Eephus,2
Slurve,1


### B. Show HRs given up by zone and pitch, show top 5 most common

In [30]:
%%sql
SELECT ZONE, pitch_name, count(*) HRs
FROM LastPitchRays
where events = 'home_run'
group by ZONE, pitch_name
order by count(*) desc
LIMIT 5

 * sqlite:///RaysPitching.db
Done.


zone,pitch_name,HRs
5,4-Seam Fastball,13
2,4-Seam Fastball,10
5,Slider,7
3,4-Seam Fastball,6
2,Sinker,5


### C. Show HRs for each count type -> Balls/Strikes + Type of Pitcher

In [31]:
%%sql
SELECT RPS.POS, LPR.balls,lpr.strikes, count(*) HRs
FROM LastPitchRays as LPR
JOIN RaysPitchingStats as RPS ON RPS.pitcher_id = LPR.pitcher
where events = 'home_run'
group by RPS.POS, LPR.balls,lpr.strikes
order by count(*) desc

 * sqlite:///RaysPitching.db
Done.


Pos,balls,strikes,HRs
SP,0,0,20
RP,0,1,18
RP,0,0,16
RP,1,0,13
SP,1,0,12
RP,1,2,9
RP,1,1,8
RP,3,2,8
SP,1,2,8
SP,1,1,7


### D. Show Each Pitchers Most Common count to give up a HR (Min 30 IP)

In [32]:
%%sql
with hrcountpitchers as (
SELECT RPS.Name, LPR.balls,lpr.strikes, count(*) HRs
FROM LastPitchRays as LPR
JOIN RaysPitchingStats as RPS ON RPS.pitcher_id = LPR.pitcher
where events = 'home_run' and IP >= 30
group by RPS.Name, LPR.balls,lpr.strikes
),
hrcountranks as (
	SELECT 
	hcp.Name, 
	hcp.balls,
	hcp.strikes, 
	hcp.HRs,
	rank() OVER (Partition by Name order by HRs desc) hrrank
	FROM hrcountpitchers as hcp
)
SELECT ht.Name, ht.balls, ht.strikes, ht.HRs
FROM hrcountranks ht
where hrrank = 1

 * sqlite:///RaysPitching.db
Done.


Name,balls,strikes,HRs
Brooks Raley,0,0,2
Colin Poche,0,1,2
Colin Poche,1,0,2
Colin Poche,1,2,2
Corey Kluber,1,0,6
Drew Rasmussen,0,0,4
Jalen Beeks,0,1,2
Jalen Beeks,1,0,2
Jason Adam,2,0,2
Jeffrey Springs,0,0,5


## Q4. Shane McClanahan

### A. Average spin rate,  strikeouts, most popular zone ONLY USING LastPitchRays

In [36]:
%%sql
SELECT
	AVG(release_spin_rate) AvgSpinRate,
	Sum(case when events = 'strikeout' then 1 else 0 end) strikeouts,
	MAX(zones.zone) as Zone
FROM LastPitchRays as LPR
join (
	SELECT pitcher, zone, count(*) zonenum
	FROM LastPitchRays as LPR
	where player_name = 'McClanahan, Shane'
	group by pitcher, zone
	order by count(*) desc
    LIMIT 1
) zones on zones.pitcher = LPR.pitcher
where player_name = 'McClanahan, Shane'

 * sqlite:///RaysPitching.db
Done.


AvgSpinRate,strikeouts,Zone
2216.975,194,13


### B. Top pitches for each infield position where total pitches are over 5, rank them

In [35]:
%%sql
SELECT *
FROM (
	SELECT pitch_name, count(*) timeshit, 'Third' Position
	FROM LastPitchRays
	WHERE hit_location = 5 and player_name = 'McClanahan, Shane'
	group by pitch_name
	UNION
	SELECT pitch_name, count(*) timeshit, 'Short' Position
	FROM LastPitchRays
	WHERE hit_location = 6 and player_name = 'McClanahan, Shane'
	group by pitch_name
	UNION
	SELECT pitch_name, count(*) timeshit, 'Second' Position
	FROM LastPitchRays
	WHERE hit_location = 4 and player_name = 'McClanahan, Shane'
	group by pitch_name
	UNION
	SELECT pitch_name, count(*) timeshit, 'First' Position
	FROM LastPitchRays
	WHERE hit_location = 3 and player_name = 'McClanahan, Shane'
	group by pitch_name
) a
where timeshit > 4
order by timeshit desc

 * sqlite:///RaysPitching.db
Done.


pitch_name,timeshit,Position
4-Seam Fastball,24,Second
Changeup,21,Third
Changeup,19,Short
Curveball,19,Third
4-Seam Fastball,15,Short
Curveball,15,Short
Curveball,14,Second
Slider,14,Second
Changeup,13,Second
4-Seam Fastball,9,Third


### C. Show different balls/strikes as well as frequency when someone is on base 

In [37]:
%%sql 
SELECT balls, strikes, count(*) frequency
FROM LastPitchRays
WHERE (on_3b is NOT NULL or on_2b is NOT NULL or on_1b is NOT NULL)
and player_name = 'McClanahan, Shane'
group by balls, strikes
order by count(*) desc

 * sqlite:///RaysPitching.db
Done.


balls,strikes,frequency
1,2,35
2,2,31
0,0,27
0,2,26
0,1,25
1,1,19
3,2,17
2,1,11
1,0,10
2,0,6


### D. What pitch causes the lowest launch speed

In [38]:
%%sql
SELECT pitch_name, avg(launch_speed * 1.00) LaunchSpeed
FROM LastPitchRays
where player_name = 'McClanahan, Shane'
group by pitch_name
order by avg(launch_speed)
LIMIT 1

 * sqlite:///RaysPitching.db
Done.


pitch_name,LaunchSpeed
Changeup,83.57075471698111
